Conhecendo as bases enviadas

In [ ]:

import pandas as pd

file = "C:/Users/Angélica/Desktop/datathon/projeto_datathon/arquivos/BASE DE DADOS PEDE 2024 - DATATHON.xlsx"

df_2024 = pd.read_excel(file, sheet_name="PEDE2024", engine="openpyxl",)
df_2024.columns = df_2024.columns.str.strip()  # Remove espaços em branco dos nomes das colunas


df_2023 = pd.read_excel(file, sheet_name="PEDE2023", engine="openpyxl",)
df_2023.columns = df_2023.columns.str.strip()  # Remove espaços em branco dos nomes das colunas


df_2022 = pd.read_excel(file, sheet_name="PEDE2022", engine="openpyxl",)
df_2022.columns = df_2022.columns.str.strip()  # Remove espaços em branco dos nomes das colunas

# comparar dataframes 
colunas_comuns = set(df_2022.columns) & set(df_2023.columns)
print(f"Colunas Iguais 2022/2023:{colunas_comuns}")

novas_cols = set(df_2023.columns) - set(df_2022.columns)
print(f"Colunas diferentes 2022/2023:{novas_cols}")

colunas_comuns = set(df_2024.columns) & set(df_2023.columns)
print(f"Colunas Iguais 2024/2023:{colunas_comuns}")

novas_cols = set(df_2023.columns) - set(df_2024.columns)
print(f"Colunas diferentes 2024/2023:{novas_cols}")

colunas_comuns = set(df_2024.columns) & set(df_2022.columns)
print(f"Colunas Iguais 2024/2022:{colunas_comuns}")

novas_cols = set(df_2022.columns) - set(df_2024.columns)
print(f"Colunas diferentes 2024/2022:{novas_cols}")

df_2024.head(5) # encontrei colunas repetidas
df_2023.head(5) # encontrei colunas repetidas
df_2022.head(5)  # OK




In [85]:
# verificação de massa de dados
df_2024.head(5) # encontrei colunas repetidas
df_2023.head(5) # encontrei colunas repetidas
df_2022.head(5)  # OK


,RA,Fase,Turma,Nome,Ano nasc,Idade 22,Gênero,Ano ingresso,Instituição de ensino,Pedra 20,...,Inglês,Indicado,Atingiu PV,IPV,IAN,Fase ideal,Defas,Destaque IEG,Destaque IDA,Destaque IPV
0,RA-1,7,A,Aluno-1,2003,19,Menina,2016,Escola Pública,Ametista,...,6.0,Sim,Não,7.278,5.0,Fase 8 (Universitários),-1,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...
1,RA-2,7,A,Aluno-2,2005,17,Menina,2017,Rede Decisão,Ametista,...,9.7,Não,Não,6.778,10.0,Fase 7 (3º EM),0,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...
2,RA-3,7,A,Aluno-3,2005,17,Menina,2016,Rede Decisão,Ametista,...,6.9,Não,Não,7.556,10.0,Fase 7 (3º EM),0,Destaque: A sua boa entrega das lições de casa.,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Destaque: A sua boa integração aos Princípios ...
3,RA-4,7,A,Aluno-4,2005,17,Menino,2017,Rede Decisão,Ametista,...,8.7,Não,Não,5.278,10.0,Fase 7 (3º EM),0,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...
4,RA-5,7,A,Aluno-5,2005,17,Menina,2016,Rede Decisão,Ametista,...,5.7,Não,Não,7.389,10.0,Fase 7 (3º EM),0,Destaque: A sua boa entrega das lições de casa.,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...


In [ ]:
# Itens para tratamento 
# duplicidade de colunas
# equalização de campos para unir as abas da planilha
# definir schema

In [ ]:

 
import sys
import os
from loguru import logger
import pandas as pd
import joblib

# Adiciona o diretório raiz do projeto ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils import LoggerConfig, ApplicationLogger

class FileUtils:
    def __init__(self, name: str = "LeituraArquivos", caminho: str = "data/", aba: str = ""):
        """
        Inicializa a aplicação
        
        Args:
            name: Nome da aplicação
            caminho: Caminho padrão para os arquivos
        """
        # Configuração do logger
        self.config = LoggerConfig(
            app_name=name,
            log_dir=f"logs/{name.lower()}"
        )
        
        self.logger = ApplicationLogger(self.__class__.__name__, self.config)
        
        self.logger.logger.info(f"Aplicação {name} inicializada")
        self.caminho = caminho
        self.aba = aba
        
    def ler_arquivo(self) -> pd.DataFrame:
        """
        Lê um arquivo CSV ou Excel e retorna um DataFrame
        
        Args:
            caminho: Caminho do arquivo a ser lido
            
        Returns:
            DataFrame com os dados lidos
        """
        
        self.logger.log_method_call("ler_arquivo", caminho=self.caminho, aba=self.aba)

        if not os.path.exists(self.caminho):
            self.logger.logger.error(f"Arquivo não encontrado: {self.caminho} / {self.aba}") 
            raise FileNotFoundError(f"Arquivo não encontrado: {self.caminho} / {self.aba}")
        
        self.logger.logger.info(f"Iniciando leitura de itens")

        if self.caminho.endswith('.xlsx'):
            df = pd.read_excel(self.caminho)
        else:
            self.logger.logger.error(f"Formato de arquivo não suportado: {self.caminho} / {self.aba}")
            raise ValueError("Formato de arquivo não suportado. Use .xlsx ou .csv")

        self.logger.logger.info(f"Dados carregados com sucesso do arquivo {self.caminho} / {self.aba}. Shape: {df.shape}. Colunas: {df.columns.tolist()}")
        return df
    
    def save_model(self, model, filepath: str):
        """
        Salva o modelo treinado em disco.
        """
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        joblib.dump(model, filepath)
        self.logger.logger.info(f"Modelo salvo em {filepath}")

    def load_model(self, filepath: str):
        """
        Carrega um modelo treinado do disco.
        """
        if not os.path.exists(filepath):
            self.logger.logger.error(f"Modelo não encontrado em: {filepath}")
            raise FileNotFoundError(f"Modelo não encontrado em: {filepath}")
        
        model = joblib.load(filepath)
        self.logger.logger.info(f"Modelo carregado de {filepath}")
        return model


class TratamentoDados:
    def __init__(self, name: str = "TratamentoDados", df):
        """
        Inicializa a aplicação
        
        Args:
            name: Nome da aplicação
            caminho: Caminho padrão para os arquivos
        """
        # Configuração do logger
        self.config = LoggerConfig(
            app_name=name,
            log_dir=f"logs/{name.lower()}"
        )
        
        self.logger = ApplicationLogger(self.__class__.__name__, self.config)
        
        self.logger.logger.info(f"Aplicação {name} inicializada")

 
    def remover_colunas_duplicadas(self):
        """
        Remove colunas duplicadas de um DataFrame.
        Se houver colunas com o mesmo nome (ex: 'Ativo/ Inativo', 'Ativo/ Inativo.1'),
        mantém apenas a primeira ocorrência.
        """
        # Identifica colunas duplicadas (pelo nome original ou pandas sufixos)
        df = df.loc[:, ~df.columns.duplicated()]
        self.logger.logger.info(f"Colunas após remoção de duplicatas exatas: {df.shape[1]}")

 
        seen = set()
        cols_to_drop = []
        
        for i, col in enumerate(df.columns):
            nome_base = col.split('.')[0] # Remove sufixo .1, .2 gerado pelo pandas
            
          
            if nome_base in seen and col != nome_base:
                # Verifica se o conteúdo é igual ao original
                if df[nome_base].equals(df[col]):
                    cols_to_drop.append(col)
                    self.logger.logger.info(f"Coluna duplicada removida: {col} (cópia idêntica de {nome_base})")
                else:
                    self.logger.logger.warning(f"Aviso: Coluna {col} tem nome similar a {nome_base} mas conteúdo diferente. Mantida.")
                    pass
            else:
                seen.add(nome_base)
                
        if cols_to_drop:
            print(f"Removendo {len(cols_to_drop)} colunas duplicadas: {cols_to_drop}")
            df = df.drop(columns=cols_to_drop)

 
        nomes_limpos = [col.split('.')[0] for col in df.columns]
        from collections import Counter
        contagem = Counter(nomes_limpos)
    
        for coluna, qtd in contagem.items():
            if qtd > 1:
                print(f" - '{coluna}': {qtd} ocorrências")
        return df


if __name__ == "__main__":    

    leitura = FileUtils(caminho="C:\\Users\\Angélica\\Desktop\\datathon\\projeto_datathon\\arquivos\\BASE DE DADOS PEDE 2024 - DATATHON.xlsx", aba="PEDE2023")
    df = leitura.ler_arquivo() 


In [87]:
remover_colunas_duplicadas(df_2023)

Colunas após remoção de duplicatas exatas: 48
Coluna duplicada removida: Destaque IPV.1 (cópia idêntica de Destaque IPV)
Removendo 1 colunas duplicadas: ['Destaque IPV.1']
 - 'Ativo/ Inativo': 2 ocorrências


,RA,Fase,INDE 2023,Pedra 2023,Turma,Nome Anonimizado,Data de Nasc,Idade,Gênero,Ano ingresso,...,Ing,Indicado,Atingiu PV,IPV,IAN,Fase Ideal,Defasagem,Destaque IEG,Destaque IDA,Destaque IPV
0,RA-861,ALFA,9.31095,Topázio,ALFA A - G0/G1,Aluno-861,6/17/2015,8,Feminino,2023,...,NaN,NaN,NaN,8.920,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN
1,RA-862,ALFA,8.22120,Topázio,ALFA A - G0/G1,Aluno-862,5/31/2014,9,Masculino,2023,...,NaN,NaN,NaN,8.585,5.0,Fase 1 (3° e 4° ano),-1,NaN,NaN,NaN
2,RA-863,ALFA,5.92975,Quartzo,ALFA A - G0/G1,Aluno-863,2/25/2016,7,Masculino,2023,...,NaN,NaN,NaN,6.260,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN
3,RA-864,ALFA,7.03400,Ametista,ALFA A - G0/G1,Aluno-864,2015-12-03 00:00:00,1900-01-08 00:00:00,Feminino,2023,...,NaN,NaN,NaN,8.500,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN
4,RA-865,ALFA,8.15520,Topázio,ALFA A - G0/G1,Aluno-865,11/13/2014,8,Masculino,2023,...,NaN,NaN,NaN,7.915,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,RA-2,FASE 8,NaN,NaN,8E,Aluno-2,5/17/2005,18,Feminino,2017,...,NaN,NaN,NaN,NaN,10.0,Fase 8 (Universitários),0,NaN,NaN,NaN
1010,RA-1271,FASE 8,NaN,NaN,8E,Aluno-1271,11/23/2004,18,Feminino,2021,...,NaN,NaN,NaN,NaN,10.0,Fase 8 (Universitários),0,NaN,NaN,NaN
1011,RA-1272,FASE 8,NaN,NaN,8E,Aluno-1272,2004-04-11 00:00:00,1900-01-18 00:00:00,Feminino,2021,...,NaN,NaN,NaN,NaN,10.0,Fase 8 (Universitários),0,NaN,NaN,NaN
1012,RA-1273,FASE 8,NaN,NaN,8E,Aluno-1273,11/29/2005,17,Feminino,2023,...,NaN,NaN,NaN,NaN,10.0,Fase 7 (3° EM),1,NaN,NaN,NaN


In [39]:
# Analisando a quantidade de colunas duplicadas
total_cols = len(df_2024.columns)
print(f"Total de colunas carregadas: {total_cols}")

# 1. Verificar nomes EXATAMENTE iguais 
# (Pandas padrão renomeia para .1, .2, então aqui deve vir vazio se você não usou mangle_dupe_cols=True)
duplicadas_exatas = df_2024.columns[df_2024.columns.duplicated()]
print(f"Duplicatas exatas de nome: {len(duplicadas_exatas)} -> {duplicadas_exatas.tolist()}")

# 2. Verificar duplicatas "semânticas" (considerando os sufixos .1, .2 que o Pandas crua)
nomes_limpos = [col.split('.')[0] for col in df_2024.columns]
from collections import Counter
contagem = Counter(nomes_limpos)

print("\nColunas que aparecem mais de uma vez (originais + renomeadas pelo Pandas):")
for coluna, qtd in contagem.items():
    if qtd > 1:
        print(f" - '{coluna}': {qtd} ocorrências")

Total de colunas carregadas: 50
Duplicatas exatas de nome: 0 -> []

Colunas que aparecem mais de uma vez (originais + renomeadas pelo Pandas):
 - 'Ativo/ Inativo': 2 ocorrências


In [93]:

 
import sys
import os
from loguru import logger
import pandas as pd
import joblib

# Adiciona o diretório raiz do projeto ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils import LoggerConfig, ApplicationLogger

class FileUtils:
    def __init__(self, name: str = "LeituraArquivos", caminho: str = "data/", aba: str = ""):
        """
        Inicializa a aplicação
        
        Args:
            name: Nome da aplicação
            caminho: Caminho padrão para os arquivos
        """
        # Configuração do logger
        self.config = LoggerConfig(
            app_name=name,
            log_dir=f"logs/{name.lower()}"
        )
        
        self.logger = ApplicationLogger(self.__class__.__name__, self.config)
        
        self.logger.logger.info(f"Aplicação {name} inicializada")
        self.caminho = caminho
        self.aba = aba
        
    def ler_arquivo(self) -> pd.DataFrame:
        """
        Lê um arquivo CSV ou Excel e retorna um DataFrame
        
        Args:
            caminho: Caminho do arquivo a ser lido
            
        Returns:
            DataFrame com os dados lidos
        """
        
        self.logger.log_method_call("ler_arquivo", caminho=self.caminho, aba=self.aba)

        if not os.path.exists(self.caminho):
            self.logger.logger.error(f"Arquivo não encontrado: {self.caminho} / {self.aba}") 
            raise FileNotFoundError(f"Arquivo não encontrado: {self.caminho} / {self.aba}")
        
        self.logger.logger.info(f"Iniciando leitura de itens")

        if self.caminho.endswith('.xlsx'):
            df = pd.read_excel(self.caminho)
        else:
            self.logger.logger.error(f"Formato de arquivo não suportado: {self.caminho} / {self.aba}")
            raise ValueError("Formato de arquivo não suportado. Use .xlsx ou .csv")

        self.logger.logger.info(f"Dados carregados com sucesso do arquivo {self.caminho} / {self.aba}. Shape: {df.shape}. Colunas: {df.columns.tolist()}")
        return df
    
    def save_model(self, model, filepath: str):
        """
        Salva o modelo treinado em disco.
        """
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        joblib.dump(model, filepath)
        self.logger.logger.info(f"Modelo salvo em {filepath}")

    def load_model(self, filepath: str):
        """
        Carrega um modelo treinado do disco.
        """
        if not os.path.exists(filepath):
            self.logger.logger.error(f"Modelo não encontrado em: {filepath}")
            raise FileNotFoundError(f"Modelo não encontrado em: {filepath}")
        
        model = joblib.load(filepath)
        self.logger.logger.info(f"Modelo carregado de {filepath}")
        return model


class TratamentoDados:
    def __init__(self, name: str = "TratamentoDados", df: pd.DataFrame()):
        """
        Inicializa a aplicação
        
        Args:
            name: Nome da aplicação
            caminho: Caminho padrão para os arquivos
        """
        # Configuração do logger
        self.config = LoggerConfig(
            app_name=name,
            log_dir=f"logs/{name.lower()}"
        )
        
        self.logger = ApplicationLogger(self.__class__.__name__, self.config)
        
        self.logger.logger.info(f"Aplicação {name} inicializada")

 
    def remover_colunas_duplicadas(self):
        """
        Remove colunas duplicadas de um DataFrame.
        Se houver colunas com o mesmo nome (ex: 'Ativo/ Inativo', 'Ativo/ Inativo.1'),
        mantém apenas a primeira ocorrência.
        """
        # Identifica colunas duplicadas (pelo nome original ou pandas sufixos)
        df = df.loc[:, ~df.columns.duplicated()]
        self.logger.logger.info(f"Colunas após remoção de duplicatas exatas: {df.shape[1]}")

 
        seen = set()
        cols_to_drop = []
        
        for i, col in enumerate(df.columns):
            nome_base = col.split('.')[0] # Remove sufixo .1, .2 gerado pelo pandas
            
          
            if nome_base in seen and col != nome_base:
                # Verifica se o conteúdo é igual ao original
                if df[nome_base].equals(df[col]):
                    cols_to_drop.append(col)
                    self.logger.logger.info(f"Coluna duplicada removida: {col} (cópia idêntica de {nome_base})")
                else:
                    self.logger.logger.warning(f"Aviso: Coluna {col} tem nome similar a {nome_base} mas conteúdo diferente. Mantida.")
                    pass
            else:
                seen.add(nome_base)
                
        if cols_to_drop:
            print(f"Removendo {len(cols_to_drop)} colunas duplicadas: {cols_to_drop}")
            df = df.drop(columns=cols_to_drop)

 
        nomes_limpos = [col.split('.')[0] for col in df.columns]
        from collections import Counter
        contagem = Counter(nomes_limpos)
    
        for coluna, qtd in contagem.items():
            if qtd > 1:
                print(f" - '{coluna}': {qtd} ocorrências")
        return df


if __name__ == "__main__":    

    leitura = FileUtils(caminho="C:\\Users\\Angélica\\Desktop\\datathon\\projeto_datathon\\arquivos\\BASE DE DADOS PEDE 2024 - DATATHON.xlsx", aba="PEDE2023")
    df = leitura.ler_arquivo() 


SyntaxError: parameter without a default follows parameter with a default (3354214625.py, line 82)